In [1]:
# !wget https://raw.githubusercontent.com/TensorSpeech/TensorFlowTTS/master/examples/hifigan/conf/hifigan.v2.yaml

In [2]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [3]:
import sys

SOURCE_DIR = os.path.dirname(os.path.dirname(os.path.abspath(__name__)))
sys.path.insert(0, SOURCE_DIR)

In [4]:
import malaya_speech
import malaya_speech.train
from malaya_speech.train.model import melgan, hifigan
from malaya_speech.train.model import stft
import malaya_speech.config
from malaya_speech.train.loss import calculate_2d_loss, calculate_3d_loss
import tensorflow as tf

In [5]:
hifigan_config = malaya_speech.config.hifigan_config_v2
hifigan_config['hifigan_generator_params']['filters'] = 512

In [6]:
generator = hifigan.Generator(
    hifigan.GeneratorConfig(**hifigan_config["hifigan_generator_params"]),
    name="hifigan_generator",
)
multigenerator = hifigan.MultiGenerator(
    hifigan.GeneratorConfig(**hifigan_config["hifigan_generator_params"]),
    name="hifigan_generator",
)

In [7]:
multiperiod_discriminator = hifigan.MultiPeriodDiscriminator(
    hifigan.DiscriminatorConfig(**hifigan_config["hifigan_discriminator_params"]),
    name="hifigan_multiperiod_discriminator",
)
multiscale_discriminator = melgan.MultiScaleDiscriminator(
    melgan.DiscriminatorConfig(
        **hifigan_config["melgan_discriminator_params"],
        name="melgan_multiscale_discriminator",
    )
)

In [8]:
discriminator = hifigan.Discriminator(multiperiod_discriminator, multiscale_discriminator)

In [9]:
y = tf.placeholder(tf.float32, (None, None))
x = tf.placeholder(tf.float32, (None, None, 80))
y_hat = generator(x)
multi_y_hat = multigenerator(x)
# p_hat = discriminator(y_hat)
# p = discriminator(tf.expand_dims(y, -1))

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [10]:
y_hat, multi_y_hat

(<tf.Tensor 'hifigan_generator/sequential/activation/Tanh:0' shape=(?, ?, 1) dtype=float32>,
 <tf.Tensor 'hifigan_generator_1/sequential_1/activation_1/Tanh:0' shape=(?, ?, 1) dtype=float32>)

In [11]:
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

In [12]:
import numpy as np

In [19]:
%%time

sess.run(y_hat, feed_dict = {x: np.random.uniform(size=(1,200,80))}).shape

CPU times: user 2.04 s, sys: 151 ms, total: 2.19 s
Wall time: 421 ms


(1, 51200, 1)

In [20]:
%%time

sess.run(multi_y_hat, feed_dict = {x: np.random.uniform(size=(1,200,80))}).shape

CPU times: user 1.99 s, sys: 246 ms, total: 2.24 s
Wall time: 341 ms


(1, 51200, 1)